In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from time import time

from modules.tokenizer import Tokenizer
from modules.transformer import Transformer, TransformerConfig
from modules.data import WikipediaTokenizedDataset

TEST_EXAMPLE = """
What is a piece of text? 101. Hello, 102. (1) [2] 567890
A text is a passage of words that conveys a set of meanings to the person who is reading it. 
It’s a body of written work, in various forms and structures, that can be words, phrases and sentences that piece together a passage of written work.
To put it as simply as possible, it is a group of words. But it can come in many different forms.
A text can be written materials, such as books, magazines, newspapers, or online content. 
But it can also be other things, those that we may not associate with standard text. 
Text could be movies, scripts, paintings, songs, political cartoons, advertisements and maps. 
If we can look at something with words and sentences, explore it, find layers of meaning in it, and draw information and conclusions from it, you’re looking at a text."""

device = "mps"

tokenizer = Tokenizer.init_and_load("/Users/maksimkoltugin/Dev/huawei_LLM_test_task/weights/tokenizer/tokenizer_15k_10k_uncased.pkl")

In [2]:
transformer = Transformer.init_and_load("/Users/maksimkoltugin/Dev/huawei_LLM_test_task/weights/ckpts/transformer_uncased/ckpt_200.pt")
# transformer = Transformer.init_and_load("/Users/maksimkoltugin/Dev/huawei_LLM_test_task/weights/ckpts/transformer_uncased/model_100.pt")

transformer = transformer.to(device)

In [3]:
text = "London is"
print(text, end="")

x = tokenizer.encode(text)
x = torch.tensor([x]).to(device)

# generate!
for i in range(100):
    # forward the model to get the logits
    with torch.no_grad():
        logits = transformer(x) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat([x, xcol], dim=1)

        print(tokenizer.decode([xcol.item()]), end="")

London is ,  was
in
es which
 on( 
, and( whichthe
 in
"  of of, but with this), of,( 
 was in to)2 the. and'
 to,
 
(
, and to s- thes of
 the was at, and
 a in as
 
 of(
  ,i
, was  to to to, in, for as<pad>. and from,